In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import torch
import dlib
from skimage import io
from scipy.spatial import distance
from typing_extensions import runtime
from yolov5 import detect
from google.colab.patches import cv2_imshow
import os

In [ ]:
# !git clone https://github.com/ultralytics/yolov5  # clone
# %cd yolov5
# %pip install -qr requirements.txt  # install

# from yolov5 import utils
# display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.0-143-gd699c21 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.0/78.2 GB disk)


In [ ]:
!unzip -q /content/drive/MyDrive/train_data.zip -d ../
!unzip -q /content/drive/MyDrive/yoloproject.zip -d ../

In [ ]:
#!zip -r yoloproject.zip /content/yolov5 

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/yolov5/runs/train/exp/weights/best.pt')

In [ ]:
face_cascade = model

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
img_list=['/content/drive/MyDrive/dlib w/dima2.jpg','/content/drive/MyDrive/dlib w/gonya3.jpg','/content/drive/MyDrive/dlib w/gigachad.jpg']
sp = dlib.shape_predictor('/content/drive/MyDrive/dlib w/shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('/content/drive/MyDrive/dlib w/dlib_face_recognition_resnet_model_v1.dat')
detector = dlib.get_frontal_face_detector()

In [ ]:
def descriptor_get(img_list,sp,facerec,detector):
  descriptor_list=[]
  for face in img_list:
      img = io.imread(face)
      dets = detector(img,1)
      for k,d in enumerate(dets):
        shape = sp(img, d)
      descriptor_list.append(facerec.compute_face_descriptor(img, shape))
  return descriptor_list

In [ ]:
desc= descriptor_get(img_list,sp,facerec,detector)

In [ ]:
def dlibface(image,desc,sp,facerec,detector):
    img = io.imread(image)  
    dets = detector(img, 1)
    if not dets:
      return 'authfailed'
    for face in desc:
      for k,d in enumerate(dets):
        shape = sp(img, d)
      face_descriptor = facerec.compute_face_descriptor(img, shape)
      a = distance.euclidean(face, face_descriptor)  
      print(a)
      if a < 0.6: 
        return 'authsuccess'
    return 'authfailed'

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # get face region coordinates
    im = cv2_imshow(img)
    count+=1
    cv2.imwrite(f'/content/drive/MyDrive/photo{count}.jpg', img)
    # im = model(img)
    # im.show()
    detect.run(weights='/content/yolov5/runs/train/exp/weights/best.pt', imgsz=(640,640), conf_thres=0.25, source=f'/content/drive/MyDrive/photo{count}.jpg', save_crop=True)
    try:
      if os.path.isdir(f'/content/yolov5/runs/detect/exp{137+count}/crops'):
        result = dlibface(f'/content/yolov5/runs/detect/exp{137+count}/crops/person/photo{count}.jpg',desc,sp,facerec,detector)
        print(result)
      else:
        raise RuntimeError 
    except RuntimeError:
      print("Not detected")
      pass

In [ ]:
#TEST
#!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source '/content/drive/MyDrive/dlib w/gosha.mp4' --save-crop 
detect.run(weights='/content/yolov5/runs/train/exp/weights/best.pt', imgsz=(640,640), conf_thres=0.25, source='/content/dimachad.jpg', save_crop=True)
#display(Image(filename='/content/yolov5/runs/detect/exp15/photo.jpg'))

In [ ]:
dlibface('/content/drive/MyDrive/dlib w/dima2.jpg',desc,sp,facerec,detector)

In [ ]:
#TRAIN
!python train.py --img 640 --batch 6 --epochs 60 --data /content/drive/MyDrive/custom_data.yaml --weights yolov5s.pt --cache 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# from pycocotools.coco import COCO
# import requests

# coco = COCO('/content/datasets/coco/annotations/instances_val2017.json')
# # Specify a list of category names of interest
# catIds = coco.getCatIds(catNms=['person'])
# # Get the corresponding image ids and images using loadImgs
# imgIds = coco.getImgIds(catIds=catIds)
# images = coco.loadImgs(imgIds)

In [ ]:
# # Save the images into a local folder
# for im in images:
#     img_data = requests.get(im['coco_url']).content
#     with open('/content/drive/MyDrive/coco_person/' + im['file_name'], 'wb') as handler:
#         handler.write(img_data)